# Import & Process Data

In [1]:
# Import dependencies
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
# Read in dataset
charity_data_df = pd.read_csv("Resources/charity_data.csv")
charity_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop columns from data df that will not impact the learning model 
charity_data_df = charity_data_df.drop(columns=['EIN'])

In [4]:
# Generate list of columns with categorical variables
charity_cat = charity_data_df.dtypes[charity_data_df.dtypes == "object"].index.tolist()

In [5]:
# Check the number of unique values in each column
charity_data_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [6]:
# Determine number of each type of value in Name column
name_counts = charity_data_df.NAME.value_counts()
name_counts

PARENT BOOSTER USA INC                                1260
TOPS CLUB INC                                          765
UNITED STATES BOWLING CONGRESS INC                     700
WASHINGTON STATE UNIVERSITY                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC        408
                                                      ... 
ADVOCATES FOR LATINO ARTS & CULTURE CONSORTIUM INC       1
GLENBROOK COUNTRY CLUB                                   1
FARMERSVILLE ROTARY CLUB                                 1
ALHAMBRA HIGH SCHOOL ARTS FOUNDATION                     1
CHRISTIAN WOMENS JOB CORPS OF MIDDLE TENNESSEE           1
Name: NAME, Length: 19568, dtype: int64

In [7]:
# Determine which values to replace
replace_name_counts = list(name_counts[name_counts < 300].index)

# Replace in DataFrame
for name_count in replace_name_counts:
    charity_data_df['NAME'] = charity_data_df['NAME'].replace(name_count,'Other')

# Check to make sure binning was successful
charity_data_df['NAME'].value_counts()

Other                                              29662
PARENT BOOSTER USA INC                              1260
TOPS CLUB INC                                        765
UNITED STATES BOWLING CONGRESS INC                   700
WASHINGTON STATE UNIVERSITY                          492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC      408
PTA TEXAS CONGRESS                                   368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC        331
ALPHA PHI SIGMA                                      313
Name: NAME, dtype: int64

In [8]:
# Determine number of each type of value in Application_type column
app_counts = charity_data_df.APPLICATION_TYPE.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Determine which values to replace
replace_app_counts = list(app_counts[app_counts < 500].index)

# Replace in DataFrame
for app_count in replace_app_counts:
    charity_data_df['APPLICATION_TYPE'] = charity_data_df['APPLICATION_TYPE'].replace(app_count,'Other')

# Check to make sure binning was successful
charity_data_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Determine number of each type of value in CLASSIFICATION column
class_counts = charity_data_df.CLASSIFICATION.value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2150        1
C1248        1
C1283        1
C1820        1
C1245        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# Determine which values to replace
replace_class_counts = list(class_counts[class_counts < 500].index)

# Replace in DataFrame
for class_count in replace_class_counts:
    charity_data_df['CLASSIFICATION'] = charity_data_df['CLASSIFICATION'].replace(class_count,'Other')

# Check to make sure binning was successful
charity_data_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

# Encode, Split, & Standardize Data

In [12]:
# Create the OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_data_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_PTA TEXAS CONGRESS,NAME_SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,NAME_TOPS CLUB INC,NAME_UNITED STATES BOWLING CONGRESS INC,NAME_WASHINGTON STATE UNIVERSITY,APPLICATION_TYPE_Other,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Merge one-hot encoded features and drop the originals
charity_data_df = charity_data_df.merge(encode_df,left_index=True, right_index=True)
charity_data_df = charity_data_df.drop(charity_cat,1)
charity_data_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_Other,NAME_PARENT BOOSTER USA INC,NAME_PTA TEXAS CONGRESS,NAME_SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,NAME_TOPS CLUB INC,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [14]:
# Split preprocessed data into features and target arrays
y = charity_data_df["IS_SUCCESSFUL"].values
X = charity_data_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create Learning Model

In [16]:
# Define the model
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 5

nn_model = tf.keras.models.Sequential()

# Add first layer to the Sequential model using Keras’ Dense class
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))                         

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                540       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train and Test Neural Network

In [18]:
# Fit / train the keras model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 75us/sample - loss: 0.5950 - accuracy: 0.6908
Epoch 2/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5360 - accuracy: 0.7330
Epoch 3/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5272 - accuracy: 0.7399
Epoch 4/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5237 - accuracy: 0.7399
Epoch 5/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5212 - accuracy: 0.7406
Epoch 6/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5200 - accuracy: 0.7430
Epoch 7/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5185 - accuracy: 0.7426
Epoch 8/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5178 - accuracy: 0.7430
Epoch 9/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5171 - accuracy: 0.7438
Epoch

25724/25724 [==============================] - 1s 44us/sample - loss: 0.5084 - accuracy: 0.7492
Epoch 77/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5083 - accuracy: 0.7490
Epoch 78/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5086 - accuracy: 0.7484
Epoch 79/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5085 - accuracy: 0.7490
Epoch 80/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5085 - accuracy: 0.7492
Epoch 81/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5080 - accuracy: 0.7492
Epoch 82/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5083 - accuracy: 0.7492
Epoch 83/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5084 - accuracy: 0.7489
Epoch 84/100
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5084 - accuracy: 0.7491
Epoch 85/100
25724/25724 [======

In [19]:
# Evaluate model performance using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 0.4821 - accuracy: 0.7399
Loss: 0.5191254244879453, Accuracy: 0.7399417161941528
